In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 256, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(256)
        self.conv2 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(512)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.5)

        self.conv3 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(512)
        self.conv4 = nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(1024)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.5)

        self.conv5 = nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(1024)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout3 = nn.Dropout(0.5)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(1024, 1024)
        self.bn6 = nn.BatchNorm1d(1024)
        self.dropout4 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(1024, 512)
        self.bn7 = nn.BatchNorm1d(512)
        self.dropout5 = nn.Dropout(0.5)

        self.fc3 = nn.Linear(512, 256)
        self.bn8 = nn.BatchNorm1d(256)
        self.dropout6 = nn.Dropout(0.5)

        self.fc4 = nn.Linear(256, 128)
        self.bn9 = nn.BatchNorm1d(128)
        self.dropout7 = nn.Dropout(0.5)

        self.fc5 = nn.Linear(128, 64)
        self.bn10 = nn.BatchNorm1d(64)
        self.dropout8 = nn.Dropout(0.5)

        self.fc6 = nn.Linear(64, 7)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        x = self.dropout2(x)

        x = F.relu(self.bn5(self.conv5(x)))
        x = self.pool3(x)
        x = self.dropout3(x)

        x = self.flatten(x)

        x = F.relu(self.bn6(self.fc1(x)))
        x = self.dropout4(x)

        x = F.relu(self.bn7(self.fc2(x)))
        x = self.dropout5(x)

        x = F.relu(self.bn8(self.fc3(x)))
        x = self.dropout6(x)

        x = F.relu(self.bn9(self.fc4(x)))
        x = self.dropout7(x)

        x = F.relu(self.bn10(self.fc5(x)))
        x = self.dropout8(x)

        x = self.fc6(x)

        return F.softmax(x, dim=1)

# Instantiate the model
model = ConvNet()


In [ ]:
!pip install torchviz

In [13]:
import torch
from torchviz import make_dot

# Instantiate the model
model = ConvNet()

model.eval()

# Create a dummy input tensor with the correct dimensions
dummy_input = torch.randn(1, 1, 13, 9)  # Assuming batch size of 1, 1 channel, 13 height, 9 width

# Pass the dummy input through the model
output = model(dummy_input)

# Visualize the model
dot = make_dot(output, params=dict(model.named_parameters()), show_attrs=True, show_saved=True)
dot.render("convnet_visualization", format="png")

'convnet_visualization.png'

In [14]:
class RecurrentModel(nn.Module):
    def __init__(self):
        super(RecurrentModel, self).__init__()

        # Define LSTM layers
        self.lstm1 = nn.LSTM(input_size=9, hidden_size=512, batch_first=True, num_layers=1, dropout=0.7)
        self.lstm2 = nn.LSTM(input_size=512, hidden_size=512, batch_first=True, num_layers=1, dropout=0.7)
        self.lstm3 = nn.LSTM(input_size=512, hidden_size=512, batch_first=True, num_layers=1, dropout=0.7)

        # Define TimeDistributed layers
        self.time_distributed1 = nn.Linear(512, 128)
        self.time_distributed2 = nn.Linear(128, 64)
        self.time_distributed3 = nn.Linear(64, 32)

        # Flatten layer
        self.flatten = nn.Flatten()

        # Define Dense layer
        self.dense = nn.Linear(32 * 13, 256)

        # Output layer
        self.output_layer = nn.Linear(256, 7)

    def forward(self, x):
        # LSTM layers
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)

        # TimeDistributed layers
        x = self.time_distributed1(x)
        x = self.time_distributed2(x)
        x = self.time_distributed3(x)

        # Flatten layer
        x = self.flatten(x)

        # Dense layer
        x = self.dense(x)

        # Output layer
        x = self.output_layer(x)

        return x


In [18]:
# Assuming input shape is (batch_size, sequence_length, input_size)
input_shape = (13, 9)
model = RecurrentModel()

# Create a dummy input tensor for testing
dummy_input = torch.randn(1, input_shape[0], input_shape[1])

# Forward pass
output = model(dummy_input)

# Visualize the model
dot = make_dot(output, params=dict(model.named_parameters()))
dot.render("RecurrentModelVisualization", format="png", cleanup=True)

'RecurrentModelVisualization.png'